<a href="https://colab.research.google.com/github/BrianBizmode/aws-cdk-examples/blob/master/arxiv_papers_googlenotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Arxivから論文のアブストラクトをダウンロードします。

# 準備(pip)

In [ ]:
!pip -q install arxiv datasets
!pip -q install markdown

# Setup

In [ ]:
# @title 入力項目
# @markdown 入手したいArXiv論文概要.

category = 'cat:cs' # @param {type: "string"}
input_search_query = 'Retrieval Augmented Generation' # @param {type: "string"}
memo_tilte = 'Recent_LLM_paper'  # @param {type: "string"}
num_results = 100  # @param {type: "number"}
# date = '2010-11-05'  # @param {type: "date"}
sort_by = "SubmittedDate"  # @param ['SubmittedDate', 'Relevance']
save_path = 'notebooklm' # @param {type: "string"}

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import arxiv
import pandas as pd

In [ ]:
base_filename = "Papers_about_" + input_search_query.replace(" ", "_")
search_query = category + " " + input_search_query
sort_by = arxiv.SortCriterion.SubmittedDate if sort_by == "SubmittedDate" else arxiv.SortCriterion.Relevance

In [ ]:
import os

def get_save_path(gdrive_path, save_path):
    if os.path.exists('/content/drive'):  # Google Driveがマウントされているか確認
        return os.path.join(gdrive_path, save_path) + "/"  # Google Driveのパスを返す
    else:
        return save_path + "/"  # ローカルのパスを返す

gdrive_path = '/content/drive/My Drive/'  # Google Driveのパス
save_path = get_save_path(gdrive_path, save_path)  # 保存パスを取得
print(save_path)  # 保存パスを表示

/content/drive/My Drive/notebooklm/


# ArXivからダウンロード

In [ ]:
import arxiv
import pandas as pd
from datasets import Dataset, DatasetDict
from arxiv import UnexpectedEmptyPageError

# Arxiv APIを使用して論文を検索します。
query = search_query

# arxiv.Clientをインスタンス化します。
client = arxiv.Client()

# 検索条件を設定します。
search = arxiv.Search(
    query=query,
    max_results=100,  # 取得する論文の最大件数を指定します。
    sort_by=sort_by
)

# 検索を実行し、結果を取得します。
results = []
try:
    for result in client.results(search): # client.results()を使用
        results.append({
            "title": result.title,
            "abstract": result.summary,
            #Bibtexを生成するために必要な情報を追加
            'authors': result.authors,
            'published': result.published,
            'journal_ref': result.journal_ref,
            'entry_id': result.entry_id
        })
except UnexpectedEmptyPageError as e:
    # エラーが発生した場合、エラーメッセージと発生したURLを表示します
    print(f"Error: {e}")
    print(f"URL: {e.url}")
    # 処理を継続するか、終了するかを決定します
    # ここでは、エラーが発生したページをスキップして処理を継続します


# BibTeX形式の引用を生成する関数
def generate_bibtex(result):
    authors = " and ".join([author.name for author in result['authors']])
    title = result['title']
    year = result['published'].year
    if result['journal_ref'] is not None:
        journal = result['journal_ref']
    else:
        journal = result['entry_id']
    bibtex = (
          f"@article{{{result['entry_id']}}},\n"
          f"author = {{{authors}}},\n"
          f"title = {{{title}}},\n"
          f"journal = {{{journal}}},\n"
          f"year = {{{year}}}\n"
          "}"
      )
    return bibtex


# BibTeX形式の引用を結果に追加します。
for result in results:
    result["bibtex"] = generate_bibtex(result)

# データフレームに変換します。
df = pd.DataFrame(results)

# title, abstract, bibtexだけのDataFrameを作成します。
df = df[["title", "abstract", "bibtex"]]

# CSVファイルとして保存します。
df.to_csv("arxiv_papers.csv", index=False)

print("データセットが 'arxiv_papers.csv' として一時保存されました。")

データセットが 'arxiv_papers.csv' として一時保存されました。


In [ ]:
import pandas as pd
df = pd.read_csv("arxiv_papers.csv")

In [ ]:
import os
import markdown

# 初期
max_chars = 50000 # NotebookLMの1ファイルあたりの許容文字数
current_chars = 0
file_number = 1
html_content = ""
to_markdown = True
os.makedirs(save_path, exist_ok=True)

for index in range(len(df)):
    content = f"""# Title: {df["title"][index]}
## abstract
{df["abstract"][index]}
## bibtex
{df['bibtex'][index]}
"""
    if to_markdown == True:
        html_entry = content
        ext = ".md"
    else:
        html_entry = markdown.markdown(content)
        ext = ".html"

    entry_length = len(html_entry)

    if current_chars + entry_length > max_chars:
        # If character limit is exceeded, save current content and start a new file
        with open(f"{save_path}{base_filename}_part{file_number}{ext}", "w") as f:
            f.write(html_content)
        print(f"{base_filename}_part{file_number}{ext} saved.")

        file_number += 1
        html_content = ""  # Reset for the new file
        current_chars = 0

    html_content += html_entry
    current_chars += entry_length

# Save the remaining content to the last file
if html_content:
    with open(f"{save_path}{base_filename}_part{file_number}{ext}", "w") as f:
        f.write(html_content)
    print(f"{save_path}{file_number}{ext} saved.")

print("HTML/md content split and saved into multiple files.")

Papers_about_Retrieval_Augmented_Generation_part1.md saved.
Papers_about_Retrieval_Augmented_Generation_part2.md saved.
Papers_about_Retrieval_Augmented_Generation_part3.md saved.
arxiv_papers_part4.md saved.
HTML/md content split and saved into multiple files.


In [ ]:
import zipfile
import os

def zip_folder(folder_path, output_path):
    """Zips a folder and its contents.

    Args:
        folder_path: Path to the folder to zip.
        output_path: Path to the output zip file.
    """
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname)

# Replace 'save_path' with the actual path to your folder
zip_folder(save_path, '/content/archive.zip')

# Download the zip file
from google.colab import files
files.download("/content/archive.zip")

print(f"{save_path} zipped and downloaded as archive.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/content/drive/My Drive/notebooklm/ zipped and downloaded as archive.zip


## Gemini Flashでまとめてみる

In [ ]:
!pip install -qU google-generativeai

In [ ]:
faile_path = f"{save_path}{base_filename}_part{1}{ext}"
with open(faile_path, 'r') as file:
  text = file.read()

In [ ]:
from google.colab import userdata
API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
import google.generativeai as genai
import os

model_name = 'gemini-1.5-flash-latest'
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel(model_name)



def summarize_text(text):

  my_prompt = f"""<Inputs>
<search_query>{input_search_query}</search_query>
</Inputs>
<Instructions Structure>
1. まず、{input_search_query}に関する論文のリストを受け取ります。
2. 次に、そのリストをもとに最近の技術傾向を要約します。
3. 最後に、要約を大学生でも分かりやすいように日本語で提供します。
</Instructions Structure>
<Instructions>
以下の内容に従って、{input_search_query}に関する最近の技術傾向をまとめてください。
提供された論文のリストを読み、各論文の主要な技術的進展やトレンドを特定します。
特定したトレンドを基に、最近の技術傾向について簡潔に要約します。
最後に、その要約を日本語で記述してください。
論文のリストは以下の通りです：
<論文リスト>
{text}
</論文リスト>
</Instructions>
"""

  response = model.generate_content(my_prompt)
  return response

# 実行
summary = summarize_text(text)

# 要約の出力
print(summary.text)

## Retrieval Augmented Generation の最近の技術トレンド

提供された論文リストから、Retrieval Augmented Generation（RAG）に関する最近の技術トレンドを以下のように要約することができます。

**1. 汎用性と効率性の向上:**

* **URAvatar** は、携帯電話スキャンから現実的なアバターを作成する手法を提案し、照明条件に依存しない汎用的なアバター生成を実現しました。リアルタイムレンダリングとグローバルイルミネーションの効率的な統合によって、より自然で没入感のあるアバター体験を可能にします。
* **EgoMimic** は、人間のエゴセントリックビデオデータを活用することで、ロボットの操作スキルを向上させるイミテーションラーニングフレームワークです。人間の動きをより直接的にロボットに適用することで、複雑なタスクの学習をより効率的に行うことができます。
* **TabM** は、パラメータ効率的なアンサンブル手法により、表形式データに対する深層学習モデルの精度と効率性を向上させます。従来の深層学習モデルに比べて、少ないパラメータで高い性能を実現することが可能です。

**2. より自然で複雑な生成:**

* **DEMO** は、テキストからビデオを生成する際に、テキストエンコーディングと条件付けをそれぞれコンテンツとモーションに分解することで、より自然な動きを持つビデオを生成します。テキストの解釈と生成を強化し、静的な出力の問題を解消するアプローチです。
* **DiffPano** は、大規模なパノラマビデオテキストデータセットと球面エピポーラ対応の拡散モデルを活用することで、テキストから一貫性のある多様なパノラマ画像を生成します。360度の画像生成を可能にし、より没入感のあるVR/AR体験を提供します。
* **LingGen** は、パワーズ法マスクを使用した新しい制御可能なテキスト生成手法です。複数の言語属性を同時に制御することができ、より多様で精度の高いテキストを生成することができます。

**3. より正確なモデリングと理解:**

* **Geometric Diffusion Bridge (GDB)** は、幾何学的状態の進化を正確に予測するための新しい生成モデルフレー